# Mecanismo de Busca Semântica para Perguntas Médicas

In [3]:
# Importar dependencias
import kagglehub
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

## Passo 1: Configuração e Carregamento de Dados

In [5]:
print("Baixando o conjunto de dados MedQuad do KaggleHub...")
# Faz o download da versão mais recente do dataset
try:
    path = kagglehub.dataset_download("pythonafroz/medquad-medical-question-answer-for-ai-research")
    print(f"Dataset baixado com sucesso em: {path}")
except Exception as e:
    print(f"Erro ao baixar o dataset. Verifique sua autenticação do Kaggle. Erro: {e}")
    exit()

print("Carregando o arquivo CSV para um DataFrame pandas...")
# Carrega o dataset
try:
    df = pd.read_csv(f"{path}/medquad.csv")
    # Limpeza básica: remove linhas onde a pergunta ou a resposta estão vazias
    df.dropna(subset=['question', 'answer'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Dataset carregado. Número de pares de pergunta-resposta: {len(df)}")
except FileNotFoundError:
    print(f"Arquivo medquad.csv não encontrado no caminho: {path}")
    exit()

# Define o dispositivo (GPU se disponível, caso contrário CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilizando o dispositivo: {device}")

print("Carregando o modelo e o tokenizador da Hugging Face...")
# Modelo pré-treinado para a língua inglesa
nome_modelo = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo).to(device)

Baixando o conjunto de dados MedQuad do KaggleHub...
Dataset baixado com sucesso em: /home/codespace/.cache/kagglehub/datasets/pythonafroz/medquad-medical-question-answer-for-ai-research/versions/1
Carregando o arquivo CSV para um DataFrame pandas...
Dataset carregado. Número de pares de pergunta-resposta: 16407
Utilizando o dispositivo: cpu
Carregando o modelo e o tokenizador da Hugging Face...


## Passo 2: Função para Geração de Embeddings (Mean Pooling)

In [6]:
def generate_embeddings(texts):
    """
    Gera embeddings para uma lista de textos usando o modelo BERT.

    Args:
        texts (list of str): Uma lista de sentenças/textos.

    Returns:
        torch.Tensor: Um tensor contendo os embeddings dos textos.
    """
    # Tokeniza as sentenças e move os tensores para o dispositivo (GPU/CPU)
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Desativa o cálculo de gradientes para economizar memória e acelerar
    with torch.no_grad():
        outputs = model(**inputs)

    # Mean Pooling: Calcula a média dos embeddings dos tokens da última camada
    # para obter um único vetor que representa a sentença inteira.
    last_hidden_states = outputs.last_hidden_state
    attention_mask = inputs['attention_mask']
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
    sum_embeddings = torch.sum(last_hidden_states * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    
    mean_pooled = sum_embeddings / sum_mask
    
    # Normaliza os embeddings para otimizar o cálculo da similaridade de cossenos
    return F.normalize(mean_pooled, p=2, dim=1)

## Passo 3: Indexação - Gerando Embeddings para Todas as Perguntas do Dataset

In [7]:
print("Iniciando a indexação: gerando embeddings para todas as perguntas do dataset...")
print("Este processo pode demorar alguns minutos dependendo do hardware...")

# Converte a coluna de perguntas para uma lista
questions_list = df['question'].tolist()

# Gera os embeddings para todas as perguntas
# Para datasets muito grandes, isso deve ser feito em lotes (batches)
# Aqui, para simplicidade, passamos a lista inteira
dataset_embeddings = generate_embeddings(questions_list).cpu() # Move para a CPU para a busca

print("Indexação concluída com sucesso!")

Iniciando a indexação: gerando embeddings para todas as perguntas do dataset...
Este processo pode demorar alguns minutos dependendo do hardware...


: 

## Passo 4: Função de Busca Semântica

In [ ]:
def find_best_answer(user_question, dataset_embeddings, data_frame):
    # Gera o embedding para a pergunta do usuário
    query_embedding = generate_embeddings([user_question]).cpu()

    # Calcula a similaridade de cossenos entre a pergunta do usuário e todas as perguntas do dataset
    # A similaridade de cossenos varia de -1 (opostos) a 1 (idênticos)
    cosine_scores = torch.matmul(query_embedding, dataset_embeddings.T)

    # Encontra o índice da pergunta com a maior similaridade
    best_match_index = torch.argmax(cosine_scores).item()

    # Recupera a pergunta, a resposta e o score do DataFrame
    best_question = data_frame.iloc[best_match_index]['question']
    best_answer = data_frame.iloc[best_match_index]['answer']
    similarity_score = cosine_scores[0][best_match_index].item()

    return best_question, best_answer, similarity_score

In [ ]:
print("--- Mecanismo de Busca Semântica Médica ---")
print("Digite sua pergunta em inglês. Digite 'sair' para terminar.")

while True:
    # Permite que o usuário insira uma pergunta
    user_query = input("\nDigite 'sair' para encerrar a aplicação\nSua pergunta: ")
    
    if user_query.lower() == 'sair':
        break

    # Busca a resposta
    similar_q, answer, score = find_best_answer(user_query, dataset_embeddings, df)

    # Exibe os resultados
    print("-" * 50)
    print(f"Pergunta do Usuário: {user_query}")
    print("-" * 50)
    print(f"Pergunta Mais Similar Encontrada (Score: {score:.4f}): {similar_q}")
    print("\nResposta Encontrada:")
    print(answer)
    print("-" * 50)